In [15]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf

In [16]:
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])
bins = [-0.43, -0.18, 0, 0.18, 0.43]
group_names = ["Severely Oversold","Neutral Oversold", "Neutral Overbought","Severely Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
pricefrommean = df.meanvalue.iloc[-1].round(2)
currentzone = df.Valuation.iloc[-1]
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]

[*********************100%***********************]  1 of 1 completed


In [17]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1],
    mode = "gauge+number+delta",
    title = {'text': "Current Price and ATH Delta"},
    number = {'prefix': "$"},
    delta = {'reference': df.price[df['price']==df.price.max()].tolist()[0], "valueformat": ".0f"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/priceath.html")

In [18]:
import plotly.graph_objects as go


fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.meanavge.iloc[-1],
    mode = "number+delta",
    number = {'prefix': "$"},
    delta = {"reference": ((df.meanavge.iloc[-2])), "valueformat": ".0f"},
    title = {'text': "Meanaverage Day Change"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))


fig.show()
fig.write_html("../static/meanaverage.html")

In [19]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value =  df.price[df['price']==df.price.max()].tolist()[0],
    mode = "number",
    number = {'prefix': "$"},
    title = {'text': "Previous ATH"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

    

fig.show()
fig.write_html("../static/previousATH.html")

In [20]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = df.price.iloc[-1] - df.meanavge.iloc[-1],
    mode = "number+delta",
    number = {'prefix': "$"},
    delta = {"reference": ((df.price.iloc[-2] - df.meanavge.iloc[-2])), "valueformat": ".0f"},
    title = {'text': "Current Price Meanaverage Delta Day Change"},
    gauge = {'axis': {'range': [None, df.price[df['price']==df.price.max()].tolist()[0]]},
    'bar': {'color': "orange"}}))

fig.show()
fig.write_html("../static/price-mean.html")

In [21]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "gauge+number",
    title = {'text': "Current Valuation - Standard Deviation percent move from meanaverage", 'font': {'size': 15}},
    value = df['move%'].iloc[-1],
    domain = {'x': [0,1], 'y': [0,1]},
    gauge = {
        'axis': {'range': [-0.43, 0.43], 'tickwidth': 1, 'tickcolor': "orange"},
        'bar': {'color': "orange"},
        'bgcolor': "white",
        'borderwidth': 2,
        'bordercolor': "white",
        'steps': [
            {'range': [-0.43, -0.18], 'color': 'red'},
            {'range': [-0.18, 0], 'color': 'yellow'},
            {'range': [0, 0.18], 'color': 'blue'},
            {'range': [0.18,0.43], 'color': 'green'}],
        }))

fig.update_layout(
    font={'color': "black", 'family': "Arial"},
    xaxis={'showgrid': False, 'range':[-1,1]},
    yaxis={'showgrid': False, 'range':[0,1.11]},
    plot_bgcolor='rgba(0,0,0,0)'
    )

# fig.add_annotation(
#     ax=0,
#     ay=0.4,
#     axref='x',
#     ayref='y',
#     x=df['move%'].iloc[-1],
#     y=0.9,
#     xref='x',
#     yref='y',
#     showarrow=True,
#     arrowhead=2,
#     arrowsize=1,
#     arrowwidth=4
#     )

# Set the visibility ON
fig.update_yaxes(title='', visible=True, showticklabels=False)
# Set the visibility OFF
fig.update_xaxes(title='x', visible=False, showticklabels=False)

fig.show()
fig.write_html("../static/valuation.html")

In [22]:
import yfinance as yf
import pandas as pd

def get_ytd_returns(ticker):
    df = yf.Ticker(ticker).history(period='ytd',interval='1d',actions=False).reset_index()
    ytd_return = ((df.Close.iloc[-1]/df.Close.iloc[0])-1)
    return ytd_return

tickers = ['BTC-USD', 'GC=F', 'Cl=F', 'TLT', 'DX=F', '^RUT', '^GSPC', '^IXIC', '^DJI', 'BIGPX']
asset_names = ['Bitcoin', 'Gold', 'Oil', '20yr Treasury', 'DXY', 'Russell 2000 Index', 'S&P 500', 'NASDAQ', 'Dow Jones', '60/40']

data = [[asset_name, get_ytd_returns(ticker)*100] for asset_name, ticker in zip(asset_names, tickers)]

# Create the pandas DataFrame
data = pd.DataFrame(data, columns=['Asset', 'YTD%'])
data['YTD%'] = data['YTD%'].round(2)


In [23]:
import plotly.express as px
fig = px.bar(data, x='YTD%', y='Asset', orientation='h', text=str("YTD%"), color="YTD%",color_continuous_scale=px.colors.sequential.Oryel)
fig.layout.yaxis.tickformat = ',.%'
fig.update_layout(title_text='Year To Date Returns')
fig.update_layout(yaxis=dict(showticklabels=True))

fig.update_layout(
    margin=dict(l=4, r=100, t=70, b=20),
)

fig.write_html("../static/YTD.html")
fig.show()

In [24]:
import pandas as pd
import yfinance as yf
from datetime import timedelta, date

# Set the start and end date
start_date = date.today() - timedelta(days=30)
end_date = date.today() 

# Define the ticker list and add BTC-USD to it
# tickers_list = ['GC=F', 'Cl=F', 'TLT', 'DX=F', '^RUT', '^GSPC', '^IXIC', '^DJI', 'BIGPX']
tickers_list = [ 'DX=F', '^GSPC', '^IXIC']
tickers_list.insert(0, 'BTC-USD')

# Fetch the data and drop rows with missing values
data1 = yf.download(tickers_list, start=start_date, end=end_date)['Close'].dropna()

# Compute the rolling correlation between BTC-USD and other tickers and add the columns to the data frame
corr_cols = [col.replace(tickers_list[0], 'BTC') for col in tickers_list[1:]]
for i, col in enumerate(corr_cols):
    corr = data1.iloc[:, 0].rolling(3).corr(data1.iloc[:, i+1])
    data1[col] = corr.values

# Reset the index
data1 = data1.reset_index()

col_mapping = {
    # "BIGPX": "60/40",
    # "TLT": "20 Yr Treasury",
    # "Cl=F": "Oil",
    # "GC=F": "Gold",
    # "^DJI": "Dow Jones",
    "^GSPC": "S&P 500",
    "^IXIC": "Nasdaq",
    "DX=F": "DXY"
    # "^RUT": "Russ 2000"
}

# Rename the columns using the mapping
data1 = data1.rename(columns=col_mapping)
data1=data1.drop(columns=['BTC-USD']).dropna()
data1

[*********************100%***********************]  4 of 4 completed


,Date,DXY,S&P 500,Nasdaq
2,2023-02-01,-0.982350,0.962050,0.991779
3,2023-02-02,-0.977303,0.482114,0.444858
4,2023-02-03,-0.791171,-0.666892,-0.823904
5,2023-02-06,-0.810270,0.799149,0.806907
6,2023-02-07,-0.866968,0.690163,0.729179
7,2023-02-08,-0.700922,0.971245,0.964924
8,2023-02-09,0.995879,0.930134,0.899025
9,2023-02-10,-0.247342,0.936289,0.967393
10,2023-02-13,-0.963072,0.310047,0.775219
11,2023-02-14,-0.888419,0.697431,0.877684


In [30]:
print(data1.DXY.mean())
print(data1.Nasdaq.mean())
print(data1['S&P 500'].mean())

-0.4236093362070349
0.500437337220437
0.47346237143379


In [26]:
import plotly.graph_objs as go

# Create the figure object
fig = go.Figure()

# Loop through the columns and add a trace for each
for col in data1.columns[1:]:
    fig.add_trace(go.Line(
        x=data1['Date'],
        y=data1[col],
        name=col
    ))

# Set the title and axis labels
fig.update_layout(
    title='Bar Chart with Looping Y-Axis',
    xaxis_title='Date',
    yaxis_title='Value'
)

fig.update_traces(dict(marker_line_width=.01))
# Show the chart
fig.show()


c:\Users\16474\anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


